# Not Using Pauli Frame Update

In [627]:
from qiskit import __version__
print(__version__)

1.4.2


In [628]:
from qiskit import QuantumCircuit, ClassicalRegister
from qiskit.quantum_info import Statevector, state_fidelity, partial_trace, DensityMatrix
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit import transpile 
import numpy as np
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError
from qiskit.circuit.controlflow import IfElseOp
from qiskit.circuit.library import XGate, ZGate
import matplotlib.pyplot as plt

In [639]:
import importlib
import steane_ec_decoder2
importlib.reload(steane_ec_decoder2)
from steane_ec_decoder2 import allstab_lookup

# Function for Encoding

In [629]:
def encoding(qc: QuantumCircuit):
    theta = np.arctan(np.sqrt((np.sqrt(5) - 1) / 2))
    amp_0 = np.cos(theta/2)
    amp_1 = np.sin(theta/2)
    qc.initialize([amp_0, amp_1], 0)
    for i in range(7):
        qc.id(i)
    for i in range(4, 7):
        qc.h(i)
    qc.cx(0, 1)
    qc.cx(0, 2)
    qc.cx(6, 0)
    qc.cx(6, 1)
    qc.cx(6, 3)
    qc.cx(5, 0)
    qc.cx(5, 2)
    qc.cx(5, 3)
    qc.cx(4, 1)
    qc.cx(4, 2)
    qc.cx(4, 3)

# Functions for Stabilizer Extraction 

In [632]:
def stabilizer6(qc: QuantumCircuit, first_qubit: int, cs: ClassicalRegister):
    # Measuring XXXXIII to qubit 7
    for i in range(4):
        qc.h(first_qubit+i)
        qc.cx(first_qubit+i, first_qubit+7)
        qc.h(first_qubit+i)
        
    # Measuring IXXIXXI to qubit 8
    for i in range([1, 2, 4, 5]):
        qc.h(first_qubit+i)
        qc.cx(first_qubit+i, first_qubit+8)
        qc.h(first_qubit+i)
    
    # Measuring IIXXIXX to qubit 9
    for i in range([2, 3, 5, 6]):
        qc.h(first_qubit+i)
        qc.cx(first_qubit+i, first_qubit+9)
        qc.h(first_qubit+i)
        
    # Measuring ZZZZIII to qubit 10
    for i in range(4):
        qc.cx(first_qubit+i, first_qubit+7)

    # Measuring IZZIZZI to qubit 11
    for i in range([1, 2, 4, 5]):
        qc.cx(first_qubit+i, first_qubit+8)

    # Measuring IIZZIZZ to qubit 12
    for i in range([2, 3, 5, 6]):
        qc.cx(first_qubit+i, first_qubit+9)
        
    # Measure
    qc.measure([first_qubit+i for i in range(7, 13)], cs)

In [631]:
def flag1(qc: QuantumCircuit, first_qubit: int, c1: ClassicalRegister):
    qc.h(first_qubit+8)
    
    for i in range(4):
        qc.h(first_qubit+i)
        
    for i in range([0, 8, 1, 2, 8, 3]):
        qc.cx(first_qubit+i, first_qubit+7)
    
    for i in range(4):
        qc.h(first_qubit+i)
    
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c1)

In [635]:
def flag2(qc: QuantumCircuit, first_qubit: int, c2: ClassicalRegister):
    qc.h(first_qubit+8)
    
    for i in range([1, 2, 4, 5]):
        qc.h(first_qubit+i)
        
    for i in range([1, 8, 2, 4, 8, 5]):
        qc.cx(first_qubit+i, first_qubit+7)
    
    for i in range([1, 2, 4, 5]):
        qc.h(first_qubit+i)
        
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c2)

In [634]:
def flag3(qc: QuantumCircuit, first_qubit: int, c3: ClassicalRegister):
    qc.h(first_qubit+8)
    
    for i in range([2, 3, 5, 6]):
        qc.h(first_qubit+i)
    
    for i in range([2, 8, 3, 5, 8, 6]):
        qc.cx(first_qubit+i, first_qubit+7)
        
    for i in range([2, 3, 5, 6]):
        qc.h(first_qubit+i)
        
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c3)

In [636]:
def flag4(qc: QuantumCircuit, first_qubit: int, c4: ClassicalRegister):
    qc.h(first_qubit+8)
    
    for i in range([0, 8, 1, 2, 8, 3]):
        qc.cx(first_qubit+i, first_qubit+7)
        
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c4)

In [637]:
def flag5(qc: QuantumCircuit, first_qubit: int, c5):
    qc.h(first_qubit+8)
        
    for i in range([1, 8, 2, 4, 8, 5]):
        qc.cx(first_qubit+i, first_qubit+7)
        
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c5)

In [638]:
def flag6(qc: QuantumCircuit, first_qubit: int, c6):
    qc.h(first_qubit+8)
    
    for i in range([2, 8, 3, 5, 8, 6]):
        qc.cx(first_qubit+i, first_qubit+7)
        
    qc.h(first_qubit+8)
    qc.measure([first_qubit+7, first_qubit+8], c6)

# Function for QEC

In [640]:
def QEC(qc: QuantumCircuit, c1: ClassicalRegister, c2: ClassicalRegister, c3: ClassicalRegister, c4: ClassicalRegister, c5: ClassicalRegister, c6: ClassicalRegister, cs1: ClassicalRegister, cs2: ClassicalRegister, cs3: ClassicalRegister, cs4: ClassicalRegister, cs5: ClassicalRegister, cs6: ClassicalRegister):
    # Measure flag1 circuit
    flag1(qc, 0, c1)
    qc.reset([7,8])
    
    # Path 1, flag and syndrome are 0 for flag1
    do_nothing = QuantumCircuit(13)
    do_nothing.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    
    # Path 2, flag or syndrome are 1 for flag1
    all_stab1 = QuantumCircuit(13)
    all_stab1.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(qc, 0, cs1)
    
    first_if = IfElseOp((c1, 0), true_body=do_nothing, false_body=all_stab1)
    qc.append(first_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    
    # Path 1, flag and syndrome are 0 for flag2
    # Path 2, flag or syndrome are 1 for flag2
    all_stab2 = QuantumCircuit(13)
    all_stab2.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(qc, 0, cs2)
    
    second_if = IfElseOp((c2, 0), true_body=do_nothing, false_body=all_stab2)
    qc.append(second_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    
    # Path 1, flag and syndrome are 0 for flag3
    # Path 2, flag or syndrome are 1 for flag3
    all_stab3 = QuantumCircuit(13)
    all_stab3.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(qc, 0, cs3)
    
    third_if = IfElseOp((c3, 0), true_body=do_nothing, false_body=all_stab3)
    qc.append(third_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    
    # Path 1, flag and syndrome are 0 for flag4
    # Path 2, flag or syndrome are 1 for flag4
    all_stab4 = QuantumCircuit(13)
    all_stab4.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(qc, 0, cs4)
    
    fourth_if = IfElseOp((c4, 0), true_body=do_nothing, false_body=all_stab4)
    qc.append(fourth_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    
    # Path 1, flag and syndrome are 0 for flag5
    # Path 2, flag or syndrome are 1 for flag5
    all_stab5 = QuantumCircuit(13)
    all_stab5.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(qc, 0, cs5)
    
    fifth_if = IfElseOp((c5, 0), true_body=do_nothing, false_body=all_stab5)
    qc.append(fifth_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    
    # Path 1, flag and syndrome are 0 for flag6
    # Path 2, flag or syndrome are 1 for flag6
    all_stab6 = QuantumCircuit(13)
    all_stab6.add_register(c1, c2, c3, c4, c5, c6, cs1, cs2, cs3, cs4, cs5, cs6)
    stabilizer6(qc, 0, cs6)
    
    sixth_if = IfElseOp((c6, 0), true_body=do_nothing, false_body=all_stab6)
    qc.append(sixth_if, list(range(13)), c1[:] + c2[:] + c3[:] + c4[:] + c5[:] + c6[:] + cs1[:] + cs2[:] + cs3[:] +cs4[:] + cs5[:] + cs6[:])
    

# Encoding + 1 Round of EC